In [1]:
import socket
import sys
import platform
import os
from scapy.all import sniff, wrpcap, rdpcap
import psutil
import time

In [2]:
system = platform.system().lower()
print(system)

windows


In [3]:
PROCESS = "FC25.exe"
DURATION = 300

In [4]:
if system == "windows":
    INTERFACE = "Беспроводная сеть"
else:
    INTERFACE = "en0"

print(INTERFACE)

Беспроводная сеть


In [5]:
def get_process_connections(process_name):
    connections = []

    for proc in psutil.process_iter(['pid', 'name']):
        try:
            if proc.info['name'].lower() == process_name.lower():
                pid = proc.info['pid']
                print(f"Найден процесс {process_name} с PID {pid}")

                print(f"Все соединения процесса: {psutil.Process(pid).net_connections()}")

                for conn in psutil.Process(pid).net_connections():
                    if (conn.type == socket.SOCK_STREAM and
                        conn.status in (psutil.CONN_ESTABLISHED, psutil.CONN_SYN_SENT, psutil.CONN_SYN_RECV)):
                        connections.append({
                            'laddr': conn.laddr.ip if conn.laddr else '',
                            'raddr': conn.raddr.ip if conn.raddr else '',
                            'lport': conn.laddr.port if conn.laddr else 0,
                            'rport': conn.raddr.port if conn.raddr else 0,
                            'type': 'tcp'
                        })
                    elif conn.type == socket.SOCK_DGRAM:
                        connections.append({
                            'laddr': conn.laddr.ip if conn.laddr else '',
                            'raddr': conn.raddr.ip if conn.raddr else '',
                            'lport': conn.laddr.port if conn.laddr else 0,
                            'rport': conn.raddr.port if conn.raddr else 0,
                            'type': 'udp'
                        })
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            continue

    if not connections:
        print(f"Не найдены активные соединения для процесса {process_name}.")
    else:
        print(f"Найдено {len(connections)} соединений для {process_name}: {connections}")
    return connections

In [6]:
def create_bpf_filter(process_name):
    connections = get_process_connections(process_name)
    if not connections:
        return ""

    ip_list = set()
    port_list = set()

    for conn in connections:
        if conn['raddr']:
            ip_list.add(conn['raddr'])
        if conn['lport']:
            port_list.add(conn['lport'])
        if conn['rport']:
            port_list.add(conn['rport'])

    if not port_list:
        print("Не найдены порты для фильтрации.")
        return ""

    port_filter = " or ".join(
        f"{conn['type']} port {port}" for port in port_list for conn in connections
        if conn['lport'] == port or conn['rport'] == port
    )

    if ip_list:
        ip_filter = " or ".join(f"host {ip}" for ip in ip_list)
        return f"({port_filter}) and ({ip_filter})"
    else:
        return port_filter

In [7]:
def start_capture(interface, output_file, duration, process_name):
    packets = []
    start_time = time.time()

    def packet_callback(packet):
        packets.append(packet)
        print(f"Захвачен пакет: {packet.summary()}")

    bpf_filter = create_bpf_filter(process_name)
    if not bpf_filter:
        print("Не удалось создать начальный BPF-фильтр. Проверьте, запущен ли процесс.")
        sys.exit(1)

    print(f"Начальный BPF-фильтр: {bpf_filter}")
    print(f"Начало захвата на интерфейсе {interface} для процесса {process_name}...")

    try:
        while time.time() - start_time < duration:
            sniff(iface=interface, prn=packet_callback, filter=bpf_filter)

            new_bpf_filter = create_bpf_filter(process_name)
            if new_bpf_filter and new_bpf_filter != bpf_filter:
                print(f"Обновлен BPF-фильтр: {new_bpf_filter}")
                bpf_filter = new_bpf_filter

        wrpcap(output_file, packets)
        print(f"Захват завершен. Файл сохранен в: {output_file}")

    except Exception as e:
        print(f"Ошибка при захвате трафика: {e}")
        sys.exit(1)

In [8]:
RAW_PATH = f'raw/games'
pcap_files = [f for f in os.listdir(RAW_PATH) if f.endswith('.pcap')]
FILENAME = f"{len(pcap_files) + 1}_{ PROCESS.replace(".exe", "") }.pcap"
OUTPUT_FILE = f"{RAW_PATH}/{FILENAME}"
print(OUTPUT_FILE)

In [10]:
start_capture(INTERFACE, OUTPUT_FILE, DURATION, PROCESS)

Найден процесс FC25.exe с PID 26332
Все соединения процесса: [pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='192.168.31.172', port=51913), raddr=addr(ip='23.217.13.119', port=443), status='CLOSE_WAIT'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='192.168.31.172', port=51966), raddr=addr(ip='54.247.186.162', port=443), status='CLOSE_WAIT'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='192.168.31.172', port=52370), raddr=addr(ip='35.214.97.63', port=443), status='ESTABLISHED'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='192.168.31.172', port=52390), raddr=addr(ip='2.23.89.119', port=443), status='ESTABLISHED'), pconn(fd=-1, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='192.168.31.172', port=51933), raddr=addr(ip='2.17.145.75', port=443), status=